In [2]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Mouse C3KO-RichLi snRNA
####  2023-05-22 by Yi Zhao (Texas Heart Institute, US)
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Initiate  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
Ver <- '0'
Step <- 'PART21_Milo_Enrichment'
Project <- '2023_neoc3ko_rli'

Code_dir <- paste0('/Volumes/shire/project/', Project, '/code/mouse_v', Ver, '/')

source(paste0(Code_dir, 'src/bioinformatics.R'))
source(paste0(Code_dir, 'src/scRNAseq.R'))
source(paste0(Code_dir, 'src/scATACseq.R'))
source(paste0(Code_dir, 'mouse_v', Ver, '.helper_functions.R'))
suppressMessages(library('miloR'))

InitiateProject('Rivendell', Ver, Step, 'mouse', Project, 'shire')
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [3]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  Load data  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
full.srt <- readRDS('integrated/PART11.cardio_cells.srt.rds')
full.srt <- DietSeurat(full.srt, assays = 'RNA', scale.data = F, dimreducs = names(full.srt@reductions))

In [9]:
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
####  MiloR Process  ####
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
RunMilo <- function(seurat, k = 20, d = 50, alpha = 0.1, reduction){
        sce <- as.SingleCellExperiment(seurat)
        milo.meta <- seurat@meta.data
        milo.obj <- Milo(sce)
        reducedDim(milo.obj, "UMAP") <- reducedDim(sce, str_to_upper(reduction)) ## because Milo only uses UMAP
        milo.obj <- buildGraph(milo.obj, k = k, d = d, reduced.dim = 'SCVI')
        milo.obj <- makeNhoods(milo.obj, k = k, d = d, refined = TRUE, prop = 0.2, reduced_dims = 'SCVI')
        milo.obj <- calcNhoodDistance(milo.obj, d = d, reduced.dim = 'SCVI')
        milo.obj <- countCells(milo.obj, samples = "group1", meta.data = milo.meta)

        milo.design <- as.data.frame(xtabs(~ group2 + group1, data = milo.meta))
        milo.design <- milo.design[milo.design$Freq > 0, ]
        milo.design <- distinct(milo.design)
        rownames(milo.design) <- milo.design$group1

        milo.res <- testNhoods(milo.obj, design = ~ group2, design.df = milo.design, reduced.dim = 'SCVI')
        milo.obj <- buildNhoodGraph(milo.obj)
        return(list(milo.obj, milo.res))
}
plotNhoodGraphDA.my <- function (x, milo_res, alpha = 0.05, res_column = "logFC", ...)
{
        if (is.character(layout)) {
                if (!layout %in% names(reducedDims(x))) {
                        stop(layout, "is not in readucedDim(x) - choose a different layout")
                }
        }
        signif_res <- milo_res
        signif_res[signif_res$SpatialFDR > alpha, res_column] <- 0
        colData(x)[res_column] <- NA
        colData(x)[unlist(nhoodIndex(x)[signif_res$Nhood]), res_column] <- signif_res[, res_column]
        plotNhoodGraph(x, colour_by = res_column, ...)
}
####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [26]:
data.group <- c('c3_p2m', 'c3_p2s', 'wt_p2m', 'wt_p2s')
for (i in 1:4){
    for (m in i+1:4){
        if(m <= 4){
#            print(c(i,m))
            print(c(data.group[i],data.group[m]))
        }
#        print(data.group[i],data.group[m])
    }
}

[1] "c3_p2m" "c3_p2s"
[1] "c3_p2m" "wt_p2m"
[1] "c3_p2m" "wt_p2s"
[1] "c3_p2s" "wt_p2m"
[1] "c3_p2s" "wt_p2s"
[1] "wt_p2m" "wt_p2s"


In [28]:
Idents(full.srt) <- 'group2'
for (i in 1:4){
    for (m in i+1:4){
        if(m <= 4){
            sub.srt <- full.srt[, full.srt$group2 %in% c(data.group[i], data.group[m])]

            cell.embeddings <- sub.srt@reductions$scVI@cell.embeddings
            sub.srt@reductions$scVI <- sub.srt@reductions$sub_clean_scVI_umap
            sub.srt@reductions$scVI@cell.embeddings <- cell.embeddings[Cells(sub.srt), ]
            sub.srt@reductions$scVI@key <- 'SCVI_'

            sub.srt$Cell_type <- droplevels(sub.srt$Cell_type)
            sub.srt$Cell_state <- droplevels(sub.srt$Cell_state)
            sub.srt$group2 <- droplevels(sub.srt$group2)
            sub.srt$group1 <- droplevels(sub.srt$group1)
            #change levels: Contrl vs Atrium

            ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            ####  Milo global  ####
            ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~----
            sub.milo <- RunMilo(sub.srt, k = 10, reduction = 'sub_clean_scVI_umap')
            milo <- sub.milo
            milo[[2]]$logFC[milo[[2]]$logFC > 5] <- 5
            milo[[2]]$logFC[milo[[2]]$logFC < -5] <- -5
            p <- plotNhoodGraphDA.my(milo[[1]], milo_res = milo[[2]], alpha = 0.1, edge_alpha = 0.1, edge_color = 'grey90') +
                    scale_fill_distiller(palette = 'RdBu', limits = c(-5, 5)) +
                    theme(aspect.ratio = 1)
            p

            #mini.srt <- sub.srt[, seq(1, ncol(sub.srt), 5)]
            #mini.srt$group1 <- droplevels(mini.srt$group1)
            #mini.srt$group2 <- droplevels(mini.srt$group2)


            #sub.milo <- RunMilo(mini.srt, k = 10, d = 50, alpha = 0.3, reduction = 'sub_clean_scVI_umap')
            #milo <- sub.milo
            #milo[[2]]$logFC[milo[[2]]$logFC > 5] <- 5
            #milo[[2]]$logFC[milo[[2]]$logFC < -5] <- -5
            #p <- plotNhoodGraphDA.my(milo[[1]], milo_res = milo[[2]], alpha = 0.1, edge_alpha = 0.1, edge_color = 'grey90') +
            #        scale_fill_distiller(palette = 'RdBu', limits = c(-5, 5)) +
            #        theme(aspect.ratio = 1)
            #p

            gc()
            PlotPDF(paste0('1.cardio.umap.',data.group[i],'_vs_',data.group[m],'_clean_milo_result'), 16, 12)
            print(p)
            dev.off()

            mtx <- as.matrix(table(sub.srt$Cell_state, sub.srt$group2))
            mtx[, 1] <- mtx[, 1]/Table(sub.srt$group2)[1]
            mtx[, 2] <- mtx[, 2]/Table(sub.srt$group2)[2]
            ratio <- log2(mtx[, 2]/mtx[, 1])
            p2 <- ggplot(data.frame(Ratio = ratio, Subtype = rownames(mtx))) +
                    geom_bar(aes(x = Ratio, y = Subtype), stat = 'identity') +
                    labs(x = paste0(data.group[i],' vs ',data.group[m]), y = 'Cell subtype') +
                    theme_classic() +
                    theme(aspect.ratio = 1)
            #p2

            gc()
            PlotPDF(paste0('2.cardio.bar.',data.group[i],'_vs_',data.group[m],'_cell_composition'), 4, 5)
            print(p2)
            dev.off()

            #Table(sub.srt$group2)
            #Table(sub.srt$group1)

            ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            saveRDS(sub.milo, paste0('analysis/PART21.Cardiomyocyte_',data.group[i], '_vs_',data.group[m],'_20pct_downsample_milo_obj_result.rds'))
            ####~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        }
#        print(data.group[i],data.group[m])
    }
}

Constructing kNN graph with k:10

Checking valid object

Running refined sampling with reduced_dim

Checking meta.data validity

Counting cells in neighbourhoods

Using TMM normalisation

Performing spatial FDR correction withk-distance weighting

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Constructing kNN graph with k:10

Checking valid object

Running refined sampling with reduced_dim

Checking meta.data validity

Counting cells in neighbourhoods

Using TMM normalisation

Performing spatial FDR correction withk-distance weighting

Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.
Constructing kNN graph with k:10

Checking valid object

Running refined sampling with reduced_dim

Checking meta.data validity

Counting cells in neighbourhoods

Using TMM normalisation

Performing spatial FDR correction withk-distance weighting

Scale for fill is already present.
Adding anot